**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from class_utils.statsmodels import SMLinearRegression
from class_utils.sklearn import (
    make_pd_column_transformer, transformer_extensions
)
from class_utils.plots import ColGrid, smoothscatter, corr_heatmap
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import norm
from itertools import chain, combinations

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/w13yiyoypnmhdco/lgb_dataset.csv?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

def powerset(iterable, start=1):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(start, len(s)+1))

## Statistical Inference Using Linear Regression: An Example

In this notebook, we are going to do a little practical demonstration of how we might go about doing statistical inference using linear regression.

To start, we are going to load our dataset, which is the dataset collected for study ["Acceptance Towards LGB Persons Is an Independent Protective Factor Against Suicide on a Country Level"](https://link.springer.com/article/10.1007/s13178-020-00477-3). The response variable is the age-standardized suicide rate (the number of suicides per 100 000 population; age-standardization is to account for the different age structures of different countries, since suicide rates vary a lot with age).

The main predictor that we want to draw inferences about is countries' LGBTQ acceptance level (on scale from 1 to 10, collected using a survey; the dataset column is called `LGBTI_inclusiveness`). In order to properly judge whether the predictor is a protective factor against suicide, we need to include a number of other predictors that are known to influence the suicide rate as covariates, because some of them might be confounding variables.

For this reason, the authors collected and included variables such as fertility rate, female labor, unemployment rate, religiosity, etc. One usually needs some domain expertise and a careful literature review to determine which predictors need to be included.

In any case, this is what the dataset looks like:



In [ ]:
df = pd.read_csv("data/lgb_dataset.csv", sep=";")
df.head()

### Should We Standardize the Data or Not?

In the next step we have two options, we can either **standardize the data**  or **leave it as it is** . Depending on our choice we are going to get either **standardized**  or **unstandardized**  regression coefficients.

#### Unstandardized Coefficients

If you decide to go with the unstandardized version, your model is going to be a bit more interpretable, since all the data will be in the original units. If you get a coefficient of, let's say -0.5 for religiosity, you will know that you would get a 0.5 decrease in the suicide rate for a unit increase in religiosity. With standardized data, the coefficients will have the same interpretation, but both the predictor and the response will be in units, which you do not understand, so it will be more difficult to make judgements about the size of the effect.

#### Standardized Coefficients

The advantage of standardized coefficients, on the other hand, is that you can make comparisons among coefficients of different predictors much more easily. Since all predictors have the same scale, it is clear that the predictor with the largest coefficient has the strongest influence on the response, etc. You cannot make this kind of comparison with unstandardized coefficients.

#### Converting Unstandardized to Standardized

Fortunately, it is very easy to convert unstandardized coefficients to standardized coefficients and vice versa. The relationship is easy to derive and it is as follows:
$$
c*i = c* {i,\mathrm{standardized}} \frac{\sigma_y}{\sigma_i}
$$
where $\sigma_i$ is the standard deviation of the $i$-th predictor and $\sigma_y$ is the standard deviation of the response.

This holds for coefficients $c_{i>0}$ of the predictors, not for the constant term $c_0$ (there the relationship is a bit different) – we are not even going to list the formula for the constant term here, since there is usually no need to do the conversion for it.

### Preprocessing

Here we are going to apply our usual preprocessing steps, which do include standardization of the input data. We are also going to standardize the response variable, since that is the convention when computing standardized regression coefficients. There is no missing data in this dataset, so the `SimpleImputer` part is actually going to be redundant.



In [ ]:
categorical_inputs = []
numeric_inputs = [
    'Fertility_Rate', 'Female_Labor', 'Unemployment', 'LGBTI_inclusiveness',
    'Religiosity', 'BIP_capita', 'Gini', 'Health Spending'
]
inputs = categorical_inputs + numeric_inputs

output = "Suicides_age_standardized"

input_preproc = make_pd_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy='constant', fill_value='MISSING'),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

output_preproc = StandardScaler()

X = input_preproc.fit_transform(df[categorical_inputs+numeric_inputs])
Y = output_preproc.fit_transform(df[[output]]).reshape(-1)

### Fitting the Model

We are going to fit the initial version of our model straight away and compute the residuals since we are going to need them when checking the assumptions of linear regression.



In [ ]:
model = SMLinearRegression()
model.fit(X, Y)

y = model.predict(X)
residuals = Y - y

### Verifying the Assumptions

As you know, in order to use linear regression for inference, we need to make sure that the assumptions of linear regression are met. So let's check them one by one. To remind you, the assumptions are [[islr](#islr),[duke](#duke)]:

# **Linearity:**  there is a linear relationship between the predictors and the predicted variable;


# **No multicollinearity:**  the predictors are not too strongly correlated with each other;


# **Independence of residuals:**  the residuals (prediction errors) on individual samples are statistically independent (there is no dependence, for example, between the residuals of the subsequent values in a time series, etc.);


# **Homoscedasticity of residuals:**  the variance of the residuals is constant, i.e. it does not depend on any of the predictors, on time, etc.;


# **Normality of residuals:**  the residuals are normally distributed and their mean is to be at zero;


### 1. Linearity

Let's start with linearity then: we are going to make our customary smoothed scatter plots and check whether the relationship between each predictors and the predicted variable is at least roughly linear.



In [ ]:
g = ColGrid(df, numeric_inputs, col_wrap=3)
g.map_dataframe(smoothscatter, y=output, linreg=True);

### 2. No Multicollinearity

Recall that when checking for multicollinearity, we examine the correlation matrix.



In [ ]:
plt.figure(figsize=(10, 10))
corr_heatmap(df[categorical_inputs+numeric_inputs], annot=True);

However, this only tells us about pair-wise correlations and not about more complex relationships involving multiple variables. But as we know, we can often diagnose problems with multicollinearity by looking at the model and its results. E.g. if the standard errors are very large, that is one of the indicators that there might be a problem with multicollinearity. So let's check the standard errors of our model – we'll see that the standard errors are not very large – certainly not orders of magnitude larger than the coefficients themselves. So hopefully we are not facing a problem with multicollinearity. We could, of course, do more work to gain additional evidence – preferably also relying on our domain knowledge – but here we are just going to leave it at that for now.



In [ ]:
model.summary()

### 3. Independence of Residuals

As we know, the independence of residuals usually cannot be verified directly from data. With time series, it is at least possible to verify whether there are time-related patterns using autocorrelation plots, but here that is not the case.

However, if we trust that the individual predictors were collected as stated in the data description, then we can assume that the residuals are indeed independent – at least there seems to be no reason to think otherwise.

### 4. Homoscedasticity of Residuals

To test homoscedasticity of the residuals, we are going to plot the residuals ($y - \hat y$) against the predicted values ($\hat y$) and check whether the variance of the residuals is constant.



In [ ]:
plt.scatter(y, residuals)
plt.axhline(0, color='r')
plt.xlabel("Predicted")
plt.ylabel("Residuals")

### 5. Normality of Residuals

We check that the mean of the residuals is close to zero and plot a histogram of the residuals (with a KDE plot) against a normal distribution with the same standard deviation. That allows us to do an easy visual comparison and check whether the shape of the distribution is reasonably close to normal.



In [ ]:
sns.histplot(
    residuals, label='residuals', stat='density',
    kde=True, alpha=0.4, linewidth=0
)

scale = residuals.std()
xx = np.linspace(-scale*3, scale*3, 100)
yy = norm.pdf(xx, loc=0, scale=scale)
plt.plot(xx, yy, color='red', label='normal')

print("mean:", residuals.mean())

plt.xlabel("residuals")
plt.ylabel("density")
plt.legend()

### Interpretation of the Full Model

Now that we have checked the assumptions, let us proceed by interpreting the model. To that end we are again going to display the model summary.



In [ ]:
model.summary()

#### Standardized Coefficients

As we can see, the bottom part of the summary displays the table of coefficients. The first column contains the names of the predictors + the constant term. The second column contains the estimated coefficients – these are the standardized coefficients since we standardized our data at the beginning. What we can see by comparing all the individual coefficients is that the LGBTQ acceptance (`LGBTI_inclusiveness`) actually has a very strong influence according to this version of our model.

#### Unstandardized Coefficients

However, to make the numbers more interpretable, we'll need to convert the standardized coefficients into unstandardized first. Recall that the relationship between the two is as follows:
$$
c*i = c* {i,\mathrm{standardized}} \frac{\sigma_y}{\sigma_i}
$$
where $\sigma_i$ is the standard deviation of the $i$-th predictor and $\sigma_y$ is the standard deviation of the response.

In the cell below, we are going to extract the standard deviation computed by the standard scalers, which are part of our preprocessing pipelines for our variable of interest. We can access the standard deviations using the `scale_` attribute of the scalers. 

Note that the standard scaler uses the uncorrected sample standard deviation (i.e. it divides by $n$ instead of $n-1$, where $n$ is the number of samples, when computing it). This is what `.std()` does when you call it on a numpy array. Do not make the mistake of using the standard scaler to preprocess and then trying to use `df['LGBTI_inclusiveness'].std()` to unstandardize the coefficient. The pandas version of the `.std()` function called in this piece of code computes the corrected, rather than the uncorrected sample standard deviation (it divides by $n-1$) and so the result would not match the value actually used to standardize the data.

To make things concrete, for `LGBTI_inclusiveness` the code might look as follows:



In [ ]:
coef_standardized = model.params['LGBTI_inclusiveness']

sigma_lgbti_inclusiveness = input_preproc.transformers_[1][1].named_steps['standardscaler'].scale_[numeric_inputs.index('LGBTI_inclusiveness')]
sigma_y = output_preproc.scale_.item()

coef_unstandardized = coef_standardized * sigma_y / sigma_lgbti_inclusiveness

print("standardized coefficient:", coef_standardized)
print("unstandardized coefficient:", coef_unstandardized)

This tells us that by increasing the value of `LGBTI_inclusiveness` by one 1 unit (on the scale of 1 to 10), the number of suicides per 100,000 people is going to decrease by roughly 1.5. In Slovakia, with the population of roughly 5.447, this would work out to saving roughly 82 lives per year ($c_i$ * 5.447 *  1e6 / 1e5).

Naturally, that is an **extremely rough estimate**  – we shouldn't even really be trying to use the model this way. There are several reasons for that: the most trivial problem is that we did not take the age standardization into account, but there is a number of much more serious issues. These relationships are not perfectly linear – e.g. they are bound to saturate at some point (otherwise for some countries with particularly low values of `LGBTI_inclusiveness`, increasing it all the way to 10 might work out to having a negative number of suicides, which clearly doesn't make sense). Also, the relationship expressed by the coefficient holds on average, but that does not mean that is is going to describe the reality of every country equally well.

Finally, the linear model actually does not provide the perfect fit to the data and its predictions are not likely to be super accurate. From the standpoint of statistical inference this is not a huge problem: it means that our model does not explain the behaviour of the response variable fully, but it does explain it to some extent and the validity of the inferences is not affected. Naturally, if we had the perfect model, we could just plug in the values for all the predictors and compute exactly how many lives would be saved by adjusting any of the predictors. But with a rough linear model, this would, of course, not work nearly as well.

Note also that we have done no model selection so far – once we decide what subset of the predictors should be used, the model is going to change and the coefficients are going to change as well. We will need to update our interpretation of the model accordingly. This is another good reason for not trying to overinterpret the values of the individual coefficients.



#### Other Results

The relationships expressed by the computed coefficients may or may not be statistically significant. Column `P>|t|` indicates the p-values – if they are below the significance level (e.g. 0.05 or 0.01), then we can consider the relationship to be statistically significant. We also get a p-value for the model as a whole: this is given in the upper part of the table under `Prob (F-statistic)`. This is the p-value for the null hypothesis that all the coefficients are zero. If it is below the significance level, then we can consider the model as such to be statistically significant (even though some of the coefficients might not be).

Let's comment on at least some of the actual coefficients as well to make things that much clearer. E.g. for the `LGBTI_inclusiveness` predictor (which measures the acceptance of LGBTQ people in the society through a survey), we clearly have statistical significance and a strong negative association, i.e. the higher the acceptance, the lower the number of suicides.

The `Religiosity` predictor shows a similar trend – there is also a relatively strong negative association between it and the number of suicides. However, in this case the p-value is quite high and so, given this data, we cannot consider the observed correlation to be statistically significant.

Naturally, we could go over all the other observations in the same way. The constant gives us the "default" value for the case where all predictors = 0.

### We Have Skipped Model Selection – Let's Address That Next

The summary contains a lot of other stats such as the AIC and BIC, R-squared and adjusted R-squared, etc. These speak to the goodness of the fit and so to the quality and predictive power of the model. These are going to be very useful to us in this next section. You may have noted that so far we have been working with the full model and we have done no model selection at all. Naturally, this full model might not be the best choice. Introducing a large number of predictors increases the number of parameters and so also the complexity of the model. This, in turn, can have a negative effect on how well the model generalizes – especially if a large portion of the predictors are actually not that useful.

So let's make the effort to choose a better model by doing variable selection. As we know, there is a number of ways to actually perform variable selection, including e.g.:

* **The all-combinations approach:**  train a model for each possible combination and pick the model that is the best in terms of some measure, e.g. the BIC. The problem is that this approach can be very computationally expensive, especially if the number of predictors is large – the number of possible models is $2^p$, where $p$ is the number of predictors.
* **Forward selection:**  start from a model with just the constant and keep adding variables one by one until some rule is satisfied. The variables are picked according to some criterion, e.g. by picking the variable that is going to reduce the residual sum of squares the most, etc.
* **Backward selection:**  start from the full model and keep removing variables one by one until some rule is satisfied. The variables are again picked according to some criterion, e.g. by picking the variable with the largest p-value, etc. The terminating rule can be e.g. that all remaining p-values are below some threshold.
#### Variable Selection Methods in Scikit-Learn

* **Univariate feature selection methods:**  these methods are based on certain univariate statistical tests (e.g. dropping features with very low variance, etc.). They can usually be applied before a model is even trained, but generally speaking, they are not very powerful. Examples include:* `SelectKBest`: selects the k best features according to some scoring function (e.g. the chi-squared test between the predictor and the class for classification, the ANOVA F-value for regression, etc.);
* `SelectPercentile`: selects the top percentile of features according to some scoring function;
* ...

* **Sequential feature selection:**  essentially an equivalent of the forward selection method described above; implemented using the `SequentialFeatureSelector` class.
* **Recursive feature elimination:**  essentially an equivalent of the backward selection method described above; implemented using the `RFE` class or the `RFECV` class (which also performs cross-validation to find the optimal number of features).
* **Model-based feature selection:**  these methods are based on some machine learning model that is used to evaluate the importance of each feature (e.g. a decision tree or a random forest); implemented using the `SelectFromModel` class.
* **Custom feature selection:**  We can also select variables e.g. by manually writing a loop over all possible models, selecting the one with the best BIC, etc.
* ...
These methods available in Scikit-Learn are not super convenient for our use case because they are designed for Scikit-Learn models, while we are using linear regression as implemented in `statsmodels` – because that also computes stats such as BIC and the p-values, which we are going to use when making our selection.

We are therefore going to implement two simple variable selection methods by hand:

* **The all-combinations approach:**  selecting from all combinations of variables using the BIC;
* **The backward selection approach:**  removing variables one by one, choosing the one with the largest p-value at each step and refitting the model after each removal – until all remaining p-values are below some threshold (e.g. 0.1).
#### The All-Combinations Approach

Let's start with the all-combinations approach. This can be quite time-consuming if the number of predictors is large (there are $2^p$ possible combinations). In our case, the number of predictors is not that large, so we can afford to do this.

We are first going to generate `feature_sets`, i.e. all possible combinations of the predictors using an auxiliary function `powerset` (the powerset is a set of all subsets, so we get all combinations of sizes ranging from 1 to the total number of features).

We are then going to iterate over all these possible sets of features, fit a linear model for each and store both the model and its BIC in lists.



In [ ]:
models = []
BICs = []
feature_sets = list(powerset(inputs))

for feats in feature_sets:
    X_sel = X[list(feats)]
    model = SMLinearRegression()
    model.fit(X_sel, Y)

    models.append(model)
    BICs.append(model.bic)

In the next step, we are going to look for the model with the minimum BIC, i.e. we are going to call argmax on the `BICs` list. This will give us the index of the model with the minimum BIC. We can then use this index to retrieve the corresponding model from the `models` list.

Finally, we are going to print the feature set that corresponds to this model and also print the model summary.



In [ ]:
best_ind = np.argmin(BICs)

print("selected features:", feature_sets[best_ind])
models[best_ind].summary()

As we can see, the model selected using this method is the one that includes `Fertility_Rate`, `Unemployment`, `LGBTI_inclusiveness` and `Religiosity`.

#### The Backward Selection Approach

Next, we are going to try the same with the backward selection approach. As we mentioned, we are going to start with the full model and then remove variables one by one, choosing the one with the largest p-value at each step and refitting the model after each removal – until all remaining p-values are below some threshold (e.g. 0.05).



In [ ]:
p_threshold = 0.05
feature_set = set(inputs)

while True:
    X_sel = X[list(feature_set)]
    model = SMLinearRegression()
    model.fit(X_sel, Y)
    pvals = model.pvalues.drop('const')

    if pvals.max() < p_threshold:
        break

    worst_feature = pvals.idxmax()
    feature_set.remove(worst_feature)

print("selected features:", feature_set)
model.summary()

As you can see, this selection method happens to favour the same model as the all-combinations approach.

### Interpreting the Results

We have now selected our final model and it is time to draw some conclusions from it in earnest. What we can see is that LGBTI acceptance is a statistically significant protective factor against suicides.

In studies of this kind, it is important to be aware of possible limitations of the results. To this end, the authors of the original paper mention e.g. the following:

* While the study design tried to eliminate all possible confounders and it is clearly very unlikely that the observed relationship is caused by reverse causation in this case, to really establish causality in a realiable way, a longitudinal study would be required.


* While the relationship between LGBTI acceptance and a lower number of suicides is clear, it is not clear whether the perceived effect can in its entirety be attributed to the LGBTI population – given that societies with accepting attitudes towards LGBTI people are usually also more accepting to other minorities, part of the effect might be explained by lowering the number of suicides in other vulnerable groups as well.


* ...




### References

<a id="duke">[duke]</a> Regression Diagnostics: Testing the Assumptions of Linear Regression. <https://people.duke.edu/~rnau/testing.htm>

<a id="islr">[islr]</a> James, G., Witten, D., Hastie, T. and Tibshirani, R., 2013. An Introduction to Statistical Learning (Vol. 112, p. 18). New York: Springer.

